In [ ]:
import os
import glob

import numpy as np
import pandas as pd

# Don't install PyNIO into any environment that also has xarray
import xarray as xr

import metpy.calc as mpcalc
from metpy.units import units

from haversine import inverse_haversine, Unit

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.pyplot import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib.collections import LineCollection
import matplotlib.lines as mlines
import matplotlib.colors as mcolors
import matplotlib.patheffects as pe
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import math
import dask.dataframe as dd
from scipy.spatial import Delaunay

from uviz.utils.tools import find_TC_bbox
from uviz.datashader_tools.utils import datashader_wrapper
from uviz.plotting.utils import basin_bboxes, ssh_wsp, ssh_mslp, sshws_color, geog_features

#plt.rcParams["font.family"] = "Arial"

# Functions

In [ ]:
def find_TC(ds, basin, times, center_dist=100, unit='km'):
    """
    Tracks tropical cyclones based on basin, specific humidity, 
    and minimum SLP.
    """
    
    # Slice data by specified date/time
    data = ds.sel(time=ds.time.isin(times))
    
    # Slice data by lat/lon bounding (specific to basin)
    lons, lats = basin_bboxes(basin)
    data = data.where((data.lon >= lons[0]) & (data.lon <= lons[1]) &\
                      (data.lat >= lats[0]) & (data.lat <= lats[1]), drop=True)
    
    # Slice data where surface specific humidity > 2% or total PW >= 74
    try:
        data = data.where(data.Q.isel(lev=-1) >= 0.02, drop=True)
    except AttributeError:
        data = data.where(data.TMQ >= 74.0, drop=True)
    
    # Slice data based on minimum sea level pressure value
    data = data.isel(data.PSL.compute().argmin(dim=('time')))
    #data = data.where(data.PSL == data.PSL.values.min(), drop=True)
    
    # Coordinates for center of TC
    lats = data['lat'].values
    lons = data['lon'].values
    
    # Spread for target bbox
    lon_e = inverse_haversine((lat, lon), center_dist, Direction.EAST, unit=unit)[1]
    lon_w = inverse_haversine((lat, lon), center_dist, Direction.WEST, unit=unit)[1]
    lat_n = inverse_haversine((lat, lon), center_dist, Direction.NORTH, unit=unit)[0]
    lat_s = inverse_haversine((lat, lon), center_dist, Direction.SOUTH, unit=unit)[0]
    
    lon_range = (lon_w, lon_e)
    lat_range = (lat_s, lat_n)
    
    return lon_range, lat_range

In [ ]:
def get_subset(simulation, bbox):
    # Function to subset a dataset by target area
    ew, sn = bbox
    
    try:
        subset = simulation.sel(lon=slice(ew[0], ew[1]), lat=slice(sn[0], sn[1]))
    except:
        subset = simulation.where((simulation['lon'] <= ew[1]) & (simulation['lon'] >= ew[0]) & 
                                  (simulation['lat'] <= sn[1]) & (simulation['lat'] >= sn[0]), drop=True)
    return subset

In [ ]:
def get_pointwise_metrics(sim, bbox):
    
    subset_sim = get_subset(sim, bbox)
    
    try:
        min_slps = subset_sim.PSL.isel(subset_sim.PSL.compute().argmin(dim=('lon', 'lat')))
        max_wsps = subset_sim.U10.isel(subset_sim.U10.compute().argmax(dim=('lon', 'lat')))
    except ValueError:
        min_slps = subset_sim.PSL.sel(ncol=subset_sim.PSL.compute().argmin(dim='ncol'))
        max_wsps = subset_sim.U10.sel(ncol=subset_sim.U10.compute().argmax(dim='ncol'))
        
    #max_wsps = max_wsps.metpy.convert_units('mph')
    
    return min_slps, max_wsps

In [ ]:
def prep_tempest(filename, times):
    original_track = pd.read_csv(filename)
    filtered_ot = original_track[original_track['time'].isin(times)].reset_index(drop=True)
    filtered_ot['ss_mslp'] = filtered_ot['slp'].apply(lambda x: ssh_mslp(x))
    return filtered_ot

In [ ]:
def plot_track_comparison(original_track, parent_subset, child_subset, parent_metric, child_metric, track_name, bbox, plot_metric=None):
    # Fix this function at some point to accommodate both mslp and wsp plotting
    
    if plot_metric == None:
        raise ValueError("Must supply 'wsp' or 'mslp' as the plotting metric.")
    
    proj = ccrs.PlateCarree()
    fig, axs = plt.subplots(1, 3, figsize=(17, 10), dpi=300, subplot_kw=dict(projection=proj), layout='constrained')
    for ax in axs.ravel():
        geog_features(ax, 'florida')

    title_fs = 18
    plt_ms = 75

    # Tempest Extremes Section
    te_mslp = original_track.slp.values
    te_lons = original_track.lon.values
    te_lats = original_track.lat.values
    
    if plot_metric == 'mslp':
        sshs_cmap = [sshws_color(slp, units='pa') for slp in te_mslp]
    elif plot_metric == 'wsp':
        sshs_cmap = [sshws_color(slp, units='pa') for slp in te_mslp]
    else:
        raise ValueError("Must supply 'wsp' or 'mslp' as the plotting metric.")
        
    te_points = np.array([te_lons, te_lats]).T.reshape(-1, 1, 2)
    te_segments = np.concatenate([te_points[:-1], te_points[1:]], axis=1)
    lc = LineCollection(te_segments, colors='k', zorder=7, transform=proj, lw=0.5)
    axs[0].add_collection(lc)
    axs[0].scatter(te_lons, te_lats, c=sshs_cmap, zorder=8, edgecolors='k', lw=0.5, s=plt_ms, transform=proj)
    axs[0].set_title(f'Original {track_name} Track', fontsize=title_fs, fontweight='bold')

    # Parent Simulation
    te_times = parent_subset.time.dt.hour.isin([0, 6, 12, 18])
    
    if plot_metric == 'mslp':
        pn_slps = parent_metric.sel(time=te_times).values
        sshs_cmap = [sshws_color(slp, units='pa') for slp in pn_slps]
    elif plot_metric == 'wsp':
        pn_wsps = parent_metric.sel(time=te_times).values
        sshs_cmap = [sshws_color(wsp, units='mph') for wsp in pn_wsps]
    else:
        raise ValueError("Must supply 'wsp' or 'mslp' as the plotting metric.")
        
    pn_lons = parent_subset.lon.sel(ncol=parent_subset.PSL.compute().argmin(dim='ncol')).sel(time=te_times).values
    pn_lats = parent_subset.lat.sel(ncol=parent_subset.PSL.compute().argmin(dim='ncol')).sel(time=te_times).values

    pn_points = np.array([pn_lons, pn_lats]).T.reshape(-1, 1, 2)
    pn_segments = np.concatenate([pn_points[:-1], pn_points[1:]], axis=1)
    lc = LineCollection(pn_segments, colors='k', zorder=9, transform=proj, lw=0.5, ls='-')
    axs[1].add_collection(lc)
    axs[1].scatter(pn_lons, pn_lats, c=sshs_cmap, zorder=10, edgecolors='k', lw=0.5, s=plt_ms, transform=proj)
    axs[1].set_title('Parent Simulation', fontsize=title_fs, fontweight='bold')

    # Child Simulation
    if plot_metric == 'mslp':
        cn_slps = child_metric.sel(time=te_times).values
        sshs_cmap = [sshws_color(slp, units='pa') for slp in cn_slps]
    elif plot_metric == 'wsp':
        cn_wsps = child_metric.sel(time=te_times).values
        sshs_cmap = [sshws_color(wsp, units='mph') for wsp in cn_wsps]
    else:
        raise ValueError("Must supply 'wsp' or 'mslp' as the plotting metric.")
        
    cn_lons = child_subset.lon.sel(ncol=child_subset.PSL.compute().argmin(dim='ncol')).sel(time=te_times).values
    cn_lats = child_subset.lat.sel(ncol=child_subset.PSL.compute().argmin(dim='ncol')).sel(time=te_times).values
    
    cn_points = np.array([cn_lons, cn_lats]).T.reshape(-1, 1, 2)
    cn_segments = np.concatenate([cn_points[:-1], cn_points[1:]], axis=1)
    lc = LineCollection(cn_segments, colors='k', zorder=9, transform=proj, lw=0.5, ls='-')
    axs[2].add_collection(lc)
    axs[2].scatter(cn_lons, cn_lats, c=sshs_cmap, zorder=10, edgecolors='k', lw=0.5, s=plt_ms, transform=proj)
    axs[2].set_title('Child Simulation', fontsize=title_fs, fontweight='bold')

    # Marker properties
    mew = 0.5     # marker edge width
    mec = 'k'      # marker edge color
    ms = 8         # marker size
    #mfc           # marker face color

    td = mlines.Line2D([], [], marker='o', ms=ms, mew=mew, mec=mec, label='Tropical Depression', mfc=sshws_color(35, 'mph'), color='k', lw=0.5, ls='-')
    ts = mlines.Line2D([], [], marker='o', ms=ms, mew=mew, mec=mec, label='Tropical Storm', mfc=sshws_color(50, 'mph'), color='k', lw=0.5, ls='-')
    c1 = mlines.Line2D([], [], marker='o', ms=ms, mew=mew, mec=mec, label='Category 1', mfc=sshws_color(75, 'mph'), color='k', lw=0.5, ls='-')
    c2 = mlines.Line2D([], [], marker='o', ms=ms, mew=mew, mec=mec, label='Category 2', mfc=sshws_color(100, 'mph'), color='k', lw=0.5, ls='-')
    c3 = mlines.Line2D([], [], marker='o', ms=ms, mew=mew, mec=mec, label='Category 3', mfc=sshws_color(115, 'mph'), color='k', lw=0.5, ls='-')
    c4 = mlines.Line2D([], [], marker='o', ms=ms, mew=mew, mec=mec, label='Category 4', mfc=sshws_color(135, 'mph'), color='k', lw=0.5, ls='-')
    c5 = mlines.Line2D([], [], marker='o', ms=ms, mew=mew, mec=mec, label='Category 5', mfc=sshws_color(160, 'mph'), color='k', lw=0.5, ls='-')

    miami_coords = (25.775163, -80.208615)
    for ax in axs.ravel():
        ax.scatter(x=miami_coords[1]+360, y=miami_coords[0], zorder=6, marker='*', c='#00FF00', s=200, edgecolors='k', lw=0.5, transform=proj)
        l = ax.legend(handles = [c5, c4, c3, c2, c1, ts, td], loc='upper right', fontsize=12.5, shadow=False)
        l.set_zorder(1001)

    #fig.suptitle('With SSHWS for Simulated Tracks', fontsize=20, fontweight='bold')

    #plt.show()
    fig.savefig(os.path.join(f"../figs/new_tracks/{track_name}", f"track_comp_{track_name}.png"), bbox_inches='tight', dpi=300, transparent=False)

In [ ]:
def plot_mslp_wsp(nparent_sim, rparent_sim, nchild_sim, rchild_sim, bbox, track_name, savefig=True):
    
    times = np.linspace(0, len(nparent_sim.time.values)*3, len(nparent_sim.time.values)+1)[1:]
    
    pn_min_slps, pn_max_wsps = get_pointwise_metrics(nparent_sim, bbox)
    pr_min_slps, pr_max_wsps = get_pointwise_metrics(rparent_sim, bbox)
    cn_min_slps, cn_max_wsps = get_pointwise_metrics(nchild_sim, bbox)
    cr_min_slps, cr_max_wsps = get_pointwise_metrics(rchild_sim, bbox)
    
    fig, axs = plt.subplots(2, 1, dpi=300, figsize=(7.75, 10), sharex=True, layout='constrained') # 8, 10 (w, h) for more longer
    lw = 1.5
    titlesize = 20
    labelsize = 14
    ticksize=12

    axs[0].plot(times, pn_min_slps.values/100, label='CAM5 (Native)', c='#024588')
    axs[0].plot(times, pr_min_slps.values/100, label='CAM5 (Regridded)', c='#7a8fc2', ls='--', lw=lw)
    axs[0].plot(times, cn_min_slps.values/100, label='MPAS (Native)', c='#BD0086') 
    axs[0].plot(times, cr_min_slps.values/100, label='MPAS (Regridded)', c='#e892c4', ls='--', lw=lw)
    axs[0].set_ylabel('Pressure [mb]', fontsize=labelsize)
    axs[0].set_title(f'Minimum Sea Level Pressure', fontsize=titlesize, fontweight='bold')
    #axs[0].legend(fontsize=12, loc='lower right')

    axs[1].plot(times, pn_max_wsps, label='CAM5 (Native)', c='#024588')
    axs[1].plot(times, pr_max_wsps, label='CAM5 (Regridded)', c='#7a8fc2', ls='--', lw=lw)
    axs[1].plot(times, cn_max_wsps, label='MPAS (Native)', c='BD0086')
    axs[1].plot(times, cr_max_wsps, label='MPAS (Regridded)', c='#e892c4', ls='--', lw=lw)
    axs[1].set_ylabel('Wind Speed [mph]', fontsize=labelsize)
    axs[1].set_title(f'Maximum Surface Wind Speed', fontsize=titlesize, fontweight='bold')
    axs[1].set_xlabel('Model Lead Time [Hours]', fontsize=labelsize)
    axs[1].legend(fontsize=14, loc='upper right')

    for ax in axs.ravel():
        start, end = ax.get_xlim()
        ax.tick_params(axis='both', which='major', labelsize=ticksize)
    
    if savefig == True:
        print(f'Saving figure for {track_name}.')
        fig.savefig(os.path.join(f"../figs/new_tracks/{track_name}", f"hourly_slp_wsp_{track_name}.png"), bbox_inches='tight', dpi=300, transparent=False)
        plt.close()
    elif savefig == False:
        plt.show()

In [ ]:
def plot_regridded_precip_swaths(rparent_ds, rchild_ds, storm_name, bbox, subset=False, savefig=True):
    
    print(f"Plotting {storm_name}.")
    lons, lats = bbox
    proj=ccrs.PlateCarree()
    if subset == True:
        lons_sfl, lats_sfl = basin_bboxes('south florida')
        rparent_ds = rparent_ds.sel(lon=slice(lons_sfl[0], lons_sfl[1]), lat=slice(lats_sfl[0], lats_sfl[1]))
        rchild_ds = rchild_ds.sel(lon=slice(lons_sfl[0], lons_sfl[1]), lat=slice(lats_sfl[0], lats_sfl[1]))
    
    fig, axs = plt.subplots(2, 2, figsize=(14,12), dpi=300, subplot_kw=dict(projection=proj), layout='constrained')

    ts = 18
    alpha = 0.75
    rparent_tot = rparent_ds['tot_prect'].where(rparent_ds['tot_prect'] >= 0.0).fillna(0)
    rparent_tot.plot(ax=axs[0][0], norm=nws_norm, cmap=nws_cmap, add_colorbar=False, alpha=alpha)
    axs[0][0].set_title('CAM5 (Regridded) Total Precipitation', fontsize=ts, fontweight='bold')

    rparent_ds['max_hrly_prect'].plot(ax=axs[0][1], norm=nws_norm, cmap=nws_cmap, add_colorbar=False, alpha=alpha)
    axs[0][1].set_title('CAM5 (Regridded) Maximum Hourly Precipitation', fontsize=ts, fontweight='bold')

    rchild_tot = rchild_ds['tot_prect'].where(rchild_ds['tot_prect'] >= 0.0).fillna(0)
    rchild_tot.plot(ax=axs[1][0], norm=nws_norm, cmap=nws_cmap, add_colorbar=False, alpha=alpha)
    axs[1][0].set_title('MPAS (Regridded) Total Precipitation', fontsize=ts, fontweight='bold')

    rchild_ds['max_hrly_prect'].plot(ax=axs[1][1], norm=nws_norm, cmap=nws_cmap, add_colorbar=False, alpha=alpha)
    axs[1][1].set_title('MPAS (Regridded) Maximum Hourly Precipitation', fontsize=ts, fontweight='bold')

    for ax in axs.ravel():
        ax.set_extent([lons[0], lons[1], lats[0], lats[1]])
        ax.coastlines(resolution='10m', linewidth=1, edgecolor='#323232')
        ax.add_feature(cfeature.LAKES.with_scale('10m'), linewidth=1, facecolor='none', edgecolor='#323232', zorder=2)

    cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=nws_norm, cmap=nws_cmap), orientation='horizontal', ax=axs, ticks=levels, alpha=alpha)
    cbar.ax.tick_params(labelsize=14)
    cbar.set_label(label='Accumulated Precipitation [inches]', size=16, fontstyle='italic')

    if savefig == True:
        if subset == False:
            fig.savefig(f"../figs/new_tracks/{storm_name}/{storm_name}_regridded_precip_swath.png", bbox_inches=None, dpi=300, transparent=False)
        else:
            fig.savefig(f"../figs/new_tracks/{storm_name}/{storm_name}_regridded_precip_swath_south_florida.png", bbox_inches=None, dpi=300, 
                        transparent=False)

    plt.close()

In [ ]:
def plot_regridded_wind_swaths(rparent_ds, rchild_ds, storm_name, bbox, subset=False, savefig=True):
    
    print(f"Plotting {storm_name}.")
    lons, lats = bbox
    proj=ccrs.PlateCarree()
    
    if subset == True:
        lons_sfl, lats_sfl = basin_bboxes('south florida')
        rparent_ds = rparent_ds.sel(lon=slice(lons_sfl[0], lons_sfl[1]), lat=slice(lats_sfl[0], lats_sfl[1]))
        rchild_ds = rchild_ds.sel(lon=slice(lons_sfl[0], lons_sfl[1]), lat=slice(lats_sfl[0], lats_sfl[1]))
    
    fig, axs = plt.subplots(1, 2, figsize=(14,7), dpi=300, subplot_kw=dict(projection=proj), layout='constrained')

    ts = 18
    alpha = 0.75
    
    vmin1 = rchild_ds['max_sfc_wsp'].sel(lon=slice(lons[0], lons[1]), lat=slice(lats[0], lats[1])).values.min()
    vmax1 = rchild_ds['max_sfc_wsp'].sel(lon=slice(lons[0], lons[1]), lat=slice(lats[0], lats[1])).values.max()
    try:
        norm1 = mcolors.TwoSlopeNorm(vmin=vmin1, vcenter=74, vmax=vmax1)
    except ValueError:
        print(f"vmin: {vmin1}, vmax: {vmax1}")
        norm1 = mcolors.TwoSlopeNorm(vmin=vmin1, vcenter=38, vmax=vmax1)
    
    rparent_ds['max_sfc_wsp'].plot(ax=axs[0], norm=norm1, cmap='viridis', add_colorbar=False, alpha=alpha)
    axs[0].set_title('CAM5 (Regridded) Maximum Surface WSP', fontsize=ts, fontweight='bold')

    rchild_ds['max_sfc_wsp'].plot(ax=axs[1], norm=norm1, cmap='viridis', add_colorbar=False, alpha=alpha)
    axs[1].set_title('MPAS (Regridded) Maximum Surface WSP', fontsize=ts, fontweight='bold')

    for ax in axs.ravel():
        ax.set_extent([lons[0], lons[1], lats[0], lats[1]])
        ax.coastlines(resolution='10m', linewidth=1, edgecolor='#323232')
        ax.add_feature(cfeature.LAKES.with_scale('10m'), linewidth=1, facecolor='none', edgecolor='#323232', zorder=2)

    cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm1, cmap='viridis'), orientation='horizontal', ax=axs)
    cbar.ax.tick_params(labelsize=14)
    cbar.set_label(label='Wind Speed [mph]', size=16, fontstyle='italic')

    if savefig == True:
        print('Exporting SFC winds.')
        fig.savefig(f"../figs/new_tracks/{storm_name}/{storm_name}_regridded_sfc_wind_swaths.png", bbox_inches=None, dpi=300, transparent=False)
    plt.close()
        
    fig, axs = plt.subplots(1, 2, figsize=(14,7), dpi=300, subplot_kw=dict(projection=proj), layout='constrained')
    vmin2 = rchild_ds['max_850_wsp'].sel(lon=slice(lons[0], lons[1]), lat=slice(lats[0], lats[1])).values.min()
    vmax2 = rchild_ds['max_850_wsp'].sel(lon=slice(lons[0], lons[1]), lat=slice(lats[0], lats[1])).values.max()
    try:
        norm2 = mcolors.TwoSlopeNorm(vmin=vmin2, vcenter=100, vmax=vmax2)
    except ValueError:
        norm2 = mcolors.TwoSlopeNorm(vmin=vmin2, vcenter=38, vmax=vmax2)
        print(f"vmin: {vmin2}, vmax: {vmax2}")

    rparent_ds['max_850_wsp'].plot(ax=axs[0], norm=norm2, cmap='viridis', add_colorbar=False, alpha=alpha)
    axs[0].set_title('CAM5 (Regridded) Maximum 850 hPa WSP', fontsize=ts, fontweight='bold')
    
    rchild_ds['max_850_wsp'].plot(ax=axs[1], norm=norm2, cmap='viridis', add_colorbar=False, alpha=alpha)
    axs[1].set_title('MPAS (Regridded) Maximum 850 hPa WSP', fontsize=ts, fontweight='bold')    
    
    for ax in axs.ravel():
        ax.set_extent([lons[0], lons[1], lats[0], lats[1]])
        ax.coastlines(resolution='10m', linewidth=1, edgecolor='#323232')
        ax.add_feature(cfeature.LAKES.with_scale('10m'), linewidth=1, facecolor='none', edgecolor='#323232', zorder=2)

    cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm2, cmap='viridis'), orientation='horizontal', ax=axs)
    cbar.ax.tick_params(labelsize=14)
    cbar.set_label(label='Wind Speed [mph]', size=16, fontstyle='italic')

    if savefig == True:
        print('Exporting 850mb winds.')
        if subset == False:
            fig.savefig(f"../figs/new_tracks/{storm_name}/{storm_name}_regridded_850_wind_swaths.png", bbox_inches=None, dpi=300, transparent=False)
        else:
            fig.savefig(f"../figs/new_tracks/{storm_name}/{storm_name}_regridded_850_wind_swaths_south_florida.png", bbox_inches=None, dpi=300, 
                        transparent=False)
    plt.close()

In [ ]:
# Block to create Class with functions

# class Storm:
#     def __init__(self, folder_path):
    
#         self.h2pn_ds = 
        
#     def 

# Data

In [ ]:
tracks_dir = r"/gpfs/group/cmz5202/default/cnd5285/synth_events"
track_folders = glob.glob(os.path.join(tracks_dir, '*storm*'))
parent_folders = [os.path.join(folder, '28km') for folder in track_folders]
child_folders = [os.path.join(folder, '3km') for folder in track_folders]

In [ ]:
storms = [s[-10:] for s in track_folders]
storms

In [ ]:
# Native parent files
h2pn_files = [glob.glob(os.path.join(f, '*h2*[!nc_remap].nc')) for f in parent_folders]
h3pn_files = [glob.glob(os.path.join(f, '*h3*[!nc_remap].nc')) for f in parent_folders]
h4pn_files = [glob.glob(os.path.join(f, '*h4*[!nc_remap].nc')) for f in parent_folders]

# Regridded parent files
h2pr_files = [glob.glob(os.path.join(f, '*h2*.nc_remap.nc')) for f in parent_folders]
h3pr_files = [glob.glob(os.path.join(f, '*h3*.nc_remap.nc')) for f in parent_folders]
h4pr_files = [glob.glob(os.path.join(f, '*h4*.nc_remap.nc')) for f in parent_folders]

# Parent mesh file
# CAM mesh is the "ext" one, based on original model run. Other options are "ref" or "wat"
p_mesh_EXT = r"/gpfs/group/cmz5202/default/cnd5285/maps_and_grids/ne0np4natlanticext.ne30x4.g_scrip.nc"
p_mesh_REF = r"/gpfs/group/cmz5202/default/cnd5285/maps_and_grids/ne0np4natlanticref.ne30x4.g_scrip.nc"
p_mesh_WAT = r"/gpfs/group/cmz5202/default/cnd5285/maps_and_grids/ne0np4natlanticwat.ne30x4.g_scrip.nc"

# Native child files
h2cn_files = [glob.glob(os.path.join(f, '*h2*[!nc_remap].nc')) for f in child_folders]
h3cn_files = [glob.glob(os.path.join(f, '*h3*[!nc_remap].nc')) for f in child_folders]
h4cn_files = [glob.glob(os.path.join(f, '*h4*[!nc_remap].nc')) for f in child_folders]

# Regridded child files
h2cr_files = [glob.glob(os.path.join(f, '*h2*.nc_remap.nc')) for f in child_folders]
h3cr_files = [glob.glob(os.path.join(f, '*h3*.nc_remap.nc')) for f in child_folders]
h4cr_files = [glob.glob(os.path.join(f, '*h4*.nc_remap.nc')) for f in child_folders]

# Child mesh file
c_mesh = "/gpfs/group/cmz5202/default/cnd5285/MPAS_3km/x20.835586.florida.init.CAM.nc"

In [ ]:
# Reads in tracks, assigns to array of xarray datasets
h2pn_ds = [xr.open_mfdataset(folder, parallel=True) for folder in h2pn_files]
h3pn_ds = [xr.open_mfdataset(folder, parallel=True) for folder in h3pn_files]
h4pn_ds = [xr.open_mfdataset(folder, parallel=True) for folder in h4pn_files]

h2pr_ds = [xr.open_mfdataset(folder, parallel=True) for folder in h2pr_files]
h3pr_ds = [xr.open_mfdataset(folder, parallel=True) for folder in h3pr_files]
h4pr_ds = [xr.open_mfdataset(folder, parallel=True) for folder in h4pr_files]

h2cn_ds = [xr.open_mfdataset(folder, parallel=True) for folder in h2cn_files]
h3cn_ds = [xr.open_mfdataset(folder, parallel=True) for folder in h3cn_files]
h4cn_ds = [xr.open_mfdataset(folder, parallel=True) for folder in h4cn_files]

h2cr_ds = [xr.open_mfdataset(folder, parallel=True) for folder in h2cr_files]
h3cr_ds = [xr.open_mfdataset(folder, parallel=True) for folder in h3cr_files]
h4cr_ds = [xr.open_mfdataset(folder, parallel=True) for folder in h4cr_files]

In [ ]:
# Reads in mesh files
p_mesh_ds_EXT = xr.open_dataset(p_mesh_EXT)
p_mesh_ds_REF = xr.open_dataset(p_mesh_REF)
p_mesh_ds_WAT = xr.open_dataset(p_mesh_WAT)

c_mesh_ds = xr.open_dataset(c_mesh)

In [ ]:
for i, ds in enumerate(h3pr_ds):
    ds['WSP850'] = mpcalc.wind_speed(ds['U850'], ds['V850']).metpy.convert_units('mph')
    h3pn_ds[i]['WSP850'] = mpcalc.wind_speed(h3pn_ds[i]['U850'], h3pn_ds[i]['V850']).metpy.convert_units('mph')
    h3cr_ds[i]['WSP850'] = mpcalc.wind_speed(h3cr_ds[i]['U850'], h3cr_ds[i]['V850']).metpy.convert_units('mph')
    h3cn_ds[i]['WSP850'] = mpcalc.wind_speed(h3cn_ds[i]['U850'], h3cn_ds[i]['V850']).metpy.convert_units('mph')
    
    ds['U10'] = ds['U10'].metpy.convert_units('mph')
    h3pn_ds[i]['U10'] = h3pn_ds[i]['U10'].metpy.convert_units('mph')
    h3cr_ds[i]['U10'] = h3cr_ds[i]['U10'].metpy.convert_units('mph')
    h3cn_ds[i]['U10'] = h3cn_ds[i]['U10'].metpy.convert_units('mph')

In [ ]:
for i, ds in enumerate(h3pr_ds):
    ds['max_sfc_wsp'] = ds['U10'].max(dim='time')
    ds['max_850_wsp'] = ds['WSP850'].max(dim='time')
    
    h3pn_ds[i]['max_sfc_wsp'] = h3pn_ds[i]['U10'].max(dim='time')
    h3pn_ds[i]['max_850_wsp'] = h3pn_ds[i]['WSP850'].max(dim='time')
    
    h3cn_ds[i]['max_sfc_wsp'] = h3cn_ds[i]['U10'].max(dim='time')
    h3cn_ds[i]['max_850_wsp'] = h3cn_ds[i]['WSP850'].max(dim='time')
    
    h3cr_ds[i]['max_sfc_wsp'] = h3cr_ds[i]['U10'].max(dim='time')
    h3cr_ds[i]['max_850_wsp'] = h3cr_ds[i]['WSP850'].max(dim='time')

In [ ]:
for i, ds in enumerate(h4pr_ds):
    ds['tot_prect'] = ds['PRECT'].metpy.convert_units('inch/hour').sum(dim='time') * units('hour')
    ds['max_hrly_prect'] = ds['PRECT'].metpy.convert_units('inch/hour').max(dim='time') * units('hour')
    
    h4pn_ds[i]['tot_prect'] = h4pn_ds[i]['PRECT'].metpy.convert_units('inch/hour').sum(dim='time') * units('hour')
    h4pn_ds[i]['max_hrly_prect'] = h4pn_ds[i]['PRECT'].metpy.convert_units('inch/hour').max(dim='time') * units('hour')

    h4cr_ds[i]['tot_prect'] = h4cr_ds[i]['PRECT'].metpy.convert_units('inch/hour').sum(dim='time') * units('hour')
    h4cr_ds[i]['max_hrly_prect'] = h4cr_ds[i]['PRECT'].metpy.convert_units('inch/hour').max(dim='time') * units('hour')

    h4cn_ds[i]['tot_prect'] = h4cn_ds[i]['PRECT'].metpy.convert_units('inch/hour').sum(dim='time') * units('hour')
    h4cn_ds[i]['max_hrly_prect'] = h4cn_ds[i]['PRECT'].metpy.convert_units('inch/hour').max(dim='time') * units('hour')

# Track Comparison

In [ ]:
# Reads in Tempest Extremes tracks, filters by shared dates with model outputs
te_times = [ds.time.dt.strftime("%Y-%m-%d %H:%M:%S").values for ds in h3cr_ds]
te_track_files = [f'../tempest_extremes/{tr}_track.csv' for tr in storms]
te_dfs = list(map(prep_tempest, te_track_files, te_times))

In [ ]:
# Repeats basin bbox by number of storms for easier mapping to functions
bboxes = [basin_bboxes('florida')]*len(storms)

# Subsets data by florida bounding box (this is faster by 10 seconds than one single function (135 vs 121s) 
pn_subsets = list(map(get_subset, h3pn_ds, bboxes))
pr_subsets = list(map(get_subset, h3pr_ds, bboxes))
cr_subsets = list(map(get_subset, h3cn_ds, bboxes))
cn_subsets = list(map(get_subset, h3cr_ds, bboxes))

# Gets plotting metrics (faster than one function, 174s vs 308s)
pn_min_slps, pn_max_wsps = zip(*map(get_pointwise_metrics, h3pn_ds, bboxes))
pr_min_slps, pr_max_wsps = zip(*map(get_pointwise_metrics, h3pr_ds, bboxes))
cn_min_slps, cn_max_wsps = zip(*map(get_pointwise_metrics, h3cn_ds, bboxes))
cr_min_slps, cr_max_wsps = zip(*map(get_pointwise_metrics, h3cr_ds, bboxes))

In [ ]:
cn_min_slps.rolling.reduce(haversine??)

In [ ]:
cn_min_slps[0]

In [ ]:
# Plots track figures, outputs to png files
list(map(plot_track_comparison, te_dfs, pn_subsets, cn_subsets, pn_min_slps, cn_min_slps, storms, bboxes, ['mslp']*len(storms)))

# Timeseries Metrics (MSLP and WSP)

In [ ]:
# Repeats basin bbox by number of storms for easier mapping to functions
bboxes = [basin_bboxes('florida')]*len(storms)
list(map(plot_mslp_wsp, h3pn_ds, h3pr_ds, h3cn_ds, h3cr_ds, bboxes, storms))

In [ ]:
plot_mslp_wsp(h3pn_ds[0], h3pr_ds[0], h3cn_ds[0], h3cr_ds[0], basin_bboxes('florida'), storms[0], savefig=True)

In [ ]:
bbox = basin_bboxes('florida')
j = 0
nchild_ds = h3cn_ds[j]
nparent_ds = h3pn_ds[j]
    
times = np.linspace(0, len(nparent_ds.time.values)*3, len(nparent_ds.time.values)+1)[1:]

pn_min_slps, pn_max_wsps = get_pointwise_metrics(nparent_ds, bbox)
cn_min_slps, cn_max_wsps = get_pointwise_metrics(nchild_ds, bbox)

fig, axs = plt.subplots(1, 2, dpi=300, figsize=(20, 6.5), layout='constrained') # 8, 10 (w, h) for more longer


lw=3.0
titles = 32 #28
labels = 24 #18
ticks = 20 #16
legends = 26 #20

axs[0].plot(times, pn_min_slps.values/100, label='CAM5-SE', c='#024588', lw=lw)
#axs[0].plot(times, pr_min_slps.values/100, label='CAM5 (Regridded)', c='#7a8fc2', ls='--', lw=lw)
axs[0].plot(times, cn_min_slps.values/100, label='CAM5-MPAS', c='#BD0086', lw=lw) 
#axs[0].plot(times, cr_min_slps.values/100, label='MPAS (Regridded)', c='#e892c4', ls='--', lw=lw)
axs[0].set_ylabel('Pressure [mb]', fontsize=labels)
axs[0].set_title(f'Minimum Sea Level Pressure', fontsize=titles, fontweight='bold', loc='left')
axs[0].set_xlabel('Model Lead Time [Hours]', fontsize=labels)
axs[0].legend(fontsize=legends, loc='upper left')

axs[1].plot(times, pn_max_wsps, label='CAM5-SE', c='#024588', lw=lw)
#axs[1].plot(times, pr_max_wsps, label='CAM5 (Regridded)', c='#7a8fc2', ls='--', lw=lw)
axs[1].plot(times, cn_max_wsps, label='CAM5-MPAS', c='#BD0086', lw=lw)
#axs[1].plot(times, cr_max_wsps, label='MPAS (Regridded)', c='#e892c4', ls='--', lw=lw)
axs[1].set_ylabel('Wind Speed [mph]', fontsize=labels)
axs[1].set_title('Maximum Surface Wind Speed', fontsize=titles, fontweight='bold', loc='left')
axs[1].set_xlabel('Model Lead Time [Hours]', fontsize=labels)
#axs[1].legend(fontsize=14, loc='upper right')

for ax in axs.ravel():
    start, end = ax.get_xlim()
    ax.tick_params(axis='both', which='major', labelsize=ticks)

fig.savefig(f"../figs/{storms[j]}_hourly_slp_wsp2.png", bbox_inches='tight', dpi=300, transparent=False)
plt.close()

# OLR

In [ ]:
from bokeh.models import FixedTicker
from scipy.spatial import Delaunay

import holoviews as hv
from holoviews import opts

import geoviews as gv
import geoviews.feature as gf

import datashader as ds
from holoviews.operation.datashader import rasterize as hds_rasterize
from holoviews.operation.datashader import datashade as hds_datashade

gv.extension("bokeh","matplotlib")
hv.extension("bokeh","matplotlib")

opts.defaults(
    opts.Image(width=1200, height=600),
    opts.RGB(width=1200, height=600))

In [ ]:
hv.help(hv.Image)

In [ ]:
# Datashader allows for custom colormap, but not custom registered colormaps. Holoviews is the opposite.

# https://cimss.ssec.wisc.edu/satellite-blog/wp-content/uploads/sites/5/2017/09/GOES16_CleanWindow_Landfall-20170920_0957_1136anim.gif
def T_to_FLUT(T, unit='C'):
    if unit == 'C':
        T += 273.15
    sigma = 5.6693E-8
    olr = sigma*(T**4)
    
    return olr

# Normalized cimss scale
btemps = np.array([-110, -92.1, -92, -80, -70, -60, -50, -42, -30, -29.9, -20, -10, 0, 10, 20, 30, 40, 57])
levels = np.array(list(map(T_to_FLUT, btemps)))
fracs = levels-T_to_FLUT(-110, 'C')
fracs = fracs/fracs[-1]

flut_colors = ['#ffffff', '#ffffff', '#e6e6e6', '#000000', '#ff1a00', '#e6ff01', '#00e30e', '#010073', '#00ffff', 
               '#bebebe', '#acacac', '#999999', '#7e7e7e', '#6c6c6c', '#525252', '#404040', '#262626', '#070707']

# flut_colors = [(fracs[0], '#ffffff'), # white, -110 
#                (fracs[1], '#ffffff'), # white, -92.1 
#                (fracs[2], '#e6e6e6'), # gray, -92
#                (fracs[3], '#000000'), # black, -80
#                (fracs[4], '#ff1a00'), # red, -70
#                (fracs[5], '#e6ff01'), # yellow-ish, -60
#                (fracs[6], '#00e30e'), # green, -50
#                (fracs[7], '#010073'), # navy, -42
#                (fracs[8], '#00ffff'), # cyan, -30
#                (fracs[9], '#bebebe'), # light gray, -29.9
#                (fracs[10], '#acacac'), # gray, -20
#                (fracs[11], '#999999'), # gray, -10
#                (fracs[12], '#7e7e7e'), # gray, 0
#                (fracs[13], '#6c6c6c'), # gray, 10
#                (fracs[14], '#525252'), # gray, 20
#                (fracs[15], '#404040'), # gray, 30
#                (fracs[16], '#262626'), # gray, 40
#                (fracs[17], '#070707')] # black, 57


flut_cimss = LinearSegmentedColormap.from_list('FLUT CIMSS', list(zip(fracs, flut_colors)), N=1200)
#mpl.colormaps.register(flut_cimss, force=True)
flut_cimss

In [ ]:
def unzipMesh(x,tris,t):
    return tris[(np.abs((x[tris[:,0]])-(x[tris[:,1]])) < t) & (np.abs((x[tris[:,0]])-(x[tris[:,2]])) < t)]

def triArea(x,y,tris):
    return ((x[tris[:,1]]-x[tris[:,0]]) * (y[tris[:,2]]-y[tris[:,0]])) - ((x[tris[:,2]]-x[tris[:,0]]) * (y[tris[:,1]]-y[tris[:,0]]))

def orderCCW(x,y,tris):
    tris[triArea(x,y,tris)<0.0,:] = tris[triArea(x,y,tris)<0.0,::-1]
    return(tris)

def triangulate(vertices, x="Longitude", y="Latitude"):
    """
    Generate a triangular mesh for the given x,y,z vertices, using Delaunay triangulation.
    For large n, typically results in about double the number of triangles as vertices.
    """
    triang = Delaunay(vertices[[x,y]].values)
    print('Given', len(vertices), "vertices, created", len(triang.simplices), 'triangles.')
    
    tris_df = pd.DataFrame(triang.simplices, columns=['v0', 'v1', 'v2'])
    
    return tris_df

def createHVTriMesh(x, y, var, var_name, triangle_indices=None, n_workers=1):
    # Declare verts array
    # This is essentally an XYZ matrix (at the location of x and y, z=?) 3 X len(verts) array
    verts = np.column_stack([x, y, var])

    # Convert to pandas
    verts_df  = pd.DataFrame(verts,  columns=['Longitude', 'Latitude', var_name])
    if isinstance(triangle_indices, np.ndarray):
        tris_df   = pd.DataFrame(triangle_indices, columns=['v0', 'v1', 'v2'])
    else:
        # Creates Delaunay triangular mesh
        tris_df = triangulate(verts_df)

    # Convert to dask
    verts_ddf = dd.from_pandas(verts_df, npartitions=n_workers)
    tris_ddf = dd.from_pandas(tris_df, npartitions=n_workers)

    # Declare HoloViews element
    tri_nodes = hv.Nodes(verts_ddf, ['Longitude', 'Latitude', 'index'], [var_name])
    trimesh = hv.TriMesh((tris_ddf, tri_nodes))
    return(trimesh)

#proj = ccrs.Robinson(central_longitude=0.0)
proj = ccrs.PlateCarree(central_longitude=0.0)
lon_range, lat_range = basin_bboxes('florida')
x_range, y_range, _ = proj.transform_points(ccrs.PlateCarree(), np.array(lon_range), np.array(lat_range)).T
lon_range = tuple(x_range)
lat_range = tuple(y_range)

In [ ]:
# MPAS settings
lonCell_c = np.mod(np.rad2deg(c_mesh_ds['lonCell'].values) - 180.0, 360.0) - 180.0
latCell_c = np.rad2deg(c_mesh_ds['latCell'].values)
lonVertex_c = np.mod(np.rad2deg(c_mesh_ds['lonVertex'].values) - 180.0, 360.0) - 180.0
latVertex_c = np.rad2deg(c_mesh_ds['latVertex'].values)

tris = c_mesh_ds.cellsOnVertex.values - 1
tris_ccw = orderCCW(lonCell_c,latCell_c,tris)
tris_ccw_flat = unzipMesh(lonCell_c,tris,90.0)

xPCS_c, yPCS_c, _ = proj.transform_points(proj, lonCell_c, latCell_c).T

In [ ]:
# CAM settings
lonCell_p = np.mod(p_mesh_ds_REF.grid_center_lon.values - 180.0, 360.0) - 180.0
latCell_p = p_mesh_ds_REF.grid_center_lat.values

# Creates Delaunay triangular mesh
xPCS_p, yPCS_p, _ = proj.transform_points(proj, lonCell_p, latCell_p).T

In [ ]:
storms

In [ ]:
ts = 16
j = 0
times = h3cn_ds[j].time.values

w = 2000 #1600
h = 1600 #1200
datashader_opts=dict(aggregator='mean', precompute=True, vdim_prefix='', pixel_ratio=5, x_range=lon_range, y_range=lat_range)
plot_opts_c = dict(cmap=flut_cimss, clim=(T_to_FLUT(-110, 'C'), T_to_FLUT(57, 'C')), width=w, height=h, xaxis=None, yaxis=None,
                        colorbar=True, colorbar_position='bottom', clabel=r'Upwelling Longwave Flux [W/m^2]', 
                        fontsize=dict(title='70pt', clabel='54pt', cticks='50pt'), title='CAM5-MPAS Upwelling Longwave Flux', 
                   colorbar_opts=dict(height=50, border_line_width=3, title_text_font='arial', major_label_text_font='arial'))
coastline_opts = dict(scale='10m', line_color='#f8f8f8', line_width=2.0, width=w, height=h)
lakes_opts = dict(projection=proj, line_width=3.0, line_color='#f8f8f8')


primalVar = h3cn_ds[j]['FLUT'].isel(time=ts).values
trimesh = createHVTriMesh(xPCS_c, yPCS_c, primalVar, 'FLUT', tris_ccw_flat, n_workers=1)
rasterized = hds_rasterize(trimesh, **datashader_opts)
mpas_flut = rasterized.opts(**plot_opts_c) * gf.coastline(projection=proj).opts(**coastline_opts) #* gf.lakes(**lakes_opts).opts(scale='10m')
#mpas_flut

In [ ]:
ts = 16
j = 0
times = h3pn_ds[j].time.values


w = 2000 #1600
h = 1400 #1100
datashader_opts=dict(aggregator='mean', precompute=True, vdim_prefix='', pixel_ratio=5, x_range=lon_range, y_range=lat_range)
plot_opts_p = dict(cmap=flut_cimss, clim=(T_to_FLUT(-110, 'C'), T_to_FLUT(57, 'C')), width=w, height=h, xaxis=None, yaxis=None,
                        colorbar=False, fontsize=dict(title='70pt'), title='CAM5-SE Upwelling Longwave Flux')
coastline_opts = dict(scale='10m', line_color='#f8f8f8', line_width=2.0, width=w, height=h)
lakes_opts = dict(projection=proj, line_width=3.0, line_color='#f8f8f8')

primalVar = h3pn_ds[j]['FLUT'].isel(time=ts).values
trimesh = createHVTriMesh(xPCS_p, yPCS_p, primalVar, 'FLUT')
rasterized = hds_rasterize(trimesh, **datashader_opts)
cam_flut = rasterized.opts(**plot_opts_p) * gf.coastline(projection=proj).opts(**coastline_opts) #* gf.lakes(**lakes_opts).opts(scale='10m')
#gf.states(projection=proj, line_width=1.5, fill_color='none', line_color='#f8f8f8').opts(scale='10m')
#cam_flut

In [ ]:
olr_figure4 = hv.Layout([cam_flut, mpas_flut]).cols(1)
hv.save(olr_figure4, '../figs/storm_0236_flut2.png', dpi=300, center=False)

In [ ]:
times[0]

In [ ]:
# For GitHub repo

j = 0
times = h3cn_ds[j].time.values[2:]
times2 = h3pn_ds[j].time.values[2:]
#times = h3cn_ds[j].time.values[26:]
#times2 = h3pn_ds[j].time.values[26:]

w = 2000
h = 1400
for i, ts in enumerate(times):   
    primalVar_c = h3cn_ds[j]['FLUT'].sel(time=ts).values
    trimesh_c = createHVTriMesh(xPCS_c, yPCS_c, primalVar_c, 'FLUT', tris_ccw_flat, n_workers=1)
    rasterized_c = hds_rasterize(trimesh_c, aggregator='mean', precompute=True, vdim_prefix='', pixel_ratio=5, x_range=lon_range, y_range=lat_range)
    mpas_olr = rasterized_c.opts(cmap=flut_cimss, clim=(T_to_FLUT(-110, 'C'), T_to_FLUT(57, 'C')), width=w, height=h, xaxis=None, yaxis=None,
                            colorbar=False) *\
    gf.coastline(projection=proj).opts(scale='10m', line_color='#f8f8f8', line_width=3.0, width=w, height=h)
    
    primalVar_p = h3pn_ds[j]['FLUT'].sel(time=times2[i]).values
    trimesh_p = createHVTriMesh(xPCS_p, yPCS_p, primalVar_p, 'FLUT')
    rasterized_p = hds_rasterize(trimesh_p, aggregator='mean', precompute=True, vdim_prefix='', pixel_ratio=5, x_range=lon_range, y_range=lat_range)
    cam_olr = rasterized_p.opts(cmap=flut_cimss, clim=(T_to_FLUT(-110, 'C'), T_to_FLUT(57, 'C')), width=w, height=h, xaxis=None, yaxis=None,
                            colorbar=False) *\
    gf.coastline(projection=proj).opts(scale='10m', line_color='#f8f8f8', line_width=3.0, width=w, height=h)

    olr_fig = hv.Layout([cam_olr, mpas_olr]).cols(2)
    hv.save(olr_fig, f"../figs/new_tracks/{storms[j]}/OLR_{storms[j]}_{ts.strftime('%m%d%Y%H')}.png", dpi=300, center=False)
    print(f'{len(times)-i} figs remaining')

In [ ]:
j = 0
output_folder = f"../figs/new_tracks/{storms[j]}"

from PIL import Image

# Create the frames

imgs = glob.glob(os.path.join(output_folder, "OLR*_resized.png"))
imgs.sort()
imgs = imgs[1:-8]

frames = []
for i in imgs:
    new_frame = Image.open(i)
    frames.append(new_frame)
    
#Save into a GIF file that loops forever
duration = 150
frames[0].save(os.path.join(output_folder, f"{storms[j]}_FLUT_resized.gif"), format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=duration, loop=0)
new_frame.close()

In [ ]:
output_folder = f"../figs/new_tracks/{storms[j]}"

from PIL import Image

basewidth = 1400
imgs = glob.glob(os.path.join(output_folder, "OLR*.png"))
imgs.sort()
imgs = imgs[1:-8]

for i in imgs:
    img = Image.open(i)
    wpercent = (basewidth/float(img.size[0]))
    hsize = int((float(img.size[1])*float(wpercent)))
    img = img.resize((basewidth,hsize), Image.Resampling.LANCZOS)
    img.save(i.replace('.png', '_resized.png'))

In [ ]:
j = 0
times = h3cn_ds[j].time.values[2:-8]
times2 = h3pn_ds[j].time.values[2:-8]

for i, ts in enumerate(times):   
    primalVar_c = h3cn_ds[j]['FLUT'].sel(time=ts).values
    trimesh_c = createHVTriMesh(xPCS_c, yPCS_c, primalVar_c, 'FLUT', tris_ccw_flat, n_workers=1)
    rasterized_c = hds_rasterize(trimesh_c, aggregator='mean', precompute=True, vdim_prefix='', pixel_ratio=5, x_range=lon_range, y_range=lat_range)
    mpas_olr = rasterized_c.opts(cmap=flut_cimss, clim=(T_to_FLUT(-110, 'C'), T_to_FLUT(57, 'C')), width=2000, height=1600, xaxis=None, yaxis=None,
                            colorbar=True, colorbar_position='bottom', clabel=r'Upwelling Longwave Flux [W/m^2]', 
                            fontsize=dict(title='60pt', clabel='48pt', cticks='40pt'), title='MPAS Upwelling Longwave Flux', 
                               colorbar_opts=dict(height=50, border_line_width=3, title_text_font='arial', major_label_text_font='arial')) *\
    gf.coastline(projection=proj).opts(scale='10m', line_color='#f8f8f8', line_width=3.0, width=2000, height=1600)
    
    primalVar_p = h3pn_ds[j]['FLUT'].sel(time=times2[i]).values
    trimesh_p = createHVTriMesh(xPCS_p, yPCS_p, primalVar_p, 'FLUT')
    rasterized_p = hds_rasterize(trimesh_p, aggregator='mean', precompute=True, vdim_prefix='', pixel_ratio=5, x_range=lon_range, y_range=lat_range)
    cam_olr = rasterized_p.opts(cmap=flut_cimss, clim=(T_to_FLUT(-110, 'C'), T_to_FLUT(57, 'C')), width=2000, height=1400, xaxis=None, yaxis=None,
                            colorbar=False, fontsize=dict(title='60pt'), title='CAM5 Upwelling Longwave Flux') *\
    gf.coastline(projection=proj).opts(scale='10m', line_color='#f8f8f8', line_width=3.0, width=2000, height=1400)

    olr_fig = hv.Layout([cam_olr, mpas_olr]).cols(1)
    hv.save(olr_fig, f"../figs/new_tracks/{storms[j]}/OLR_{storms[j]}_{ts.strftime('%m%d%Y%H')}.png", dpi=300, center=False)
    print(f'{len(times)-i} figs remaining')

In [ ]:
j = 0
times = h3cn_ds[j].time.values[2:]
times2 = h3pn_ds[j].time.values[2:]

for i, ts in enumerate(times):   
    primalVar_c = h3cn_ds[j]['FLUT'].sel(time=ts).values
    trimesh_c = createHVTriMesh(xPCS_c, yPCS_c, primalVar_c, 'FLUT', tris_ccw_flat, n_workers=1)
    rasterized_c = hds_rasterize(trimesh_c, aggregator='mean', precompute=True, vdim_prefix='', pixel_ratio=5, x_range=lon_range, y_range=lat_range)
    mpas_olr = rasterized.opts(cmap=flut_cimss, clim=(T_to_FLUT(-110, 'C'), T_to_FLUT(57, 'C')), width=2000, height=1600, xaxis=None, yaxis=None,
                            colorbar=True, colorbar_position='bottom', clabel=r'Upwelling Longwave Flux [W/m^2]', 
                            fontsize=dict(title='60pt', clabel='48pt', cticks='40pt'), title='MPAS Upwelling Longwave Flux', 
                               colorbar_opts=dict(height=50, border_line_width=3, title_text_font='arial', major_label_text_font='arial')) *\
    gf.coastline(projection=proj).opts(scale='10m', line_color='#f8f8f8', line_width=2.5, width=2000, height=1600)
    
    primalVar_p = h3pn_ds[j]['FLUT'].sel(time=times2[i]).values
    trimesh_p = createHVTriMesh(xPCS_p, yPCS_p, primalVar_p, 'FLUT')
    rasterized_p = hds_rasterize(trimesh_p, aggregator='mean', precompute=True, vdim_prefix='', pixel_ratio=5, x_range=lon_range, y_range=lat_range)
    cam_olr = rasterized.opts(cmap=flut_cimss, clim=(T_to_FLUT(-110, 'C'), T_to_FLUT(57, 'C')), width=2000, height=1400, xaxis=None, yaxis=None,
                            colorbar=False, fontsize=dict(title='60pt'), title='CAM5 Upwelling Longwave Flux') *\
    gf.coastline(projection=proj).opts(scale='10m', line_color='#f8f8f8', line_width=2.5, width=2000, height=1400)

    olr_fig = hv.Layout([cam_olr, mpas_olr]).cols(1)
    hv.save(olr_fig, f"../figs/new_tracks/{storms[j]}/OLR_{storms[j]}_{ts.strftime('%m%d%Y%H')}.png", dpi=300, center=False)
    print(f'{len(times)-i} figs remaining')

In [ ]:
#times = pd.date_range('2008-08-30', '2008-09-04', freq='3H')

lon_range, lat_range = find_TC_bbox(h3cn_ds[-1], 'florida', 23, center_dist=1000)
proj = ccrs.Robinson(central_longitude=0.0)
xPCS, yPCS, _ = proj.transform_points(ccrs.PlateCarree(), lonCell, latCell).T

x_range, y_range, _ = proj.transform_points(ccrs.PlateCarree(), np.array(lon_range), np.array(lat_range)).T
lon_range = tuple(x_range)
lat_range = tuple(y_range)

j = 8
times = h3cn_ds[j].time.values

#fontsize=dict(clabel='16pt', cticks='12pt'), clabel='Upwelling Longwave Flux [W/m^2]', # height = 500 for colorbar=True

for i, ts in enumerate(times[2:]):
#for i, ts in enumerate(times[18:]):    
    #title = f'Upwelling Longwave Flux MPAS (Native Grid) {ts}'
    primalVar = h3cn_ds[j]['FLUT'].sel(time=ts).values
    trimesh = createHVTriMesh(xPCS,yPCS,tris_ccw_flat, primalVar, 'FLUT', n_workers=1)
    rasterized = hds_rasterize(trimesh, aggregator='mean', precompute=True, vdim_prefix='', pixel_ratio=5, x_range=lon_range, y_range=lat_range)
    final = rasterized.opts(toolbar=None, colorbar=False, cmap=flut_cimss, clim=(T_to_FLUT(-110, 'C'), T_to_FLUT(57, 'C')), width=800, height=600) *\
    gf.coastline(projection=proj).options(scale='10m')

    hv.save(final, f"../figs/new_tracks/{storms[j]}/MPAS_{storms[j]}_{ts.strftime('%m%d%Y%H')}_nocolorbar.png", backend='bokeh', dpi=300)
    print(f'{len(times[2:])-i} figs remaining')

In [ ]:
j = 3
output_folder = f"../figs/new_tracks/{storms[j]}"

from PIL import Image

# Create the frames

imgs = glob.glob(os.path.join(output_folder, "*MPAS*_nocolorbar.png"))
imgs.sort()
imgs = imgs[1:]

frames = []
for i in imgs:
    new_frame = Image.open(i)
    frames.append(new_frame)
    
#Save into a GIF file that loops forever
duration = 240
frames[0].save(os.path.join(output_folder, f"MPAS_{storms[j]}_FLUT_nocolorbar_{duration}.gif"), format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=duration, loop=0)
new_frame.close()

In [ ]:
def triangulate(vertices, x="Longitude", y="Latitude"):
    """
    Generate a triangular mesh for the given x,y,z vertices, using Delaunay triangulation.
    For large n, typically results in about double the number of triangles as vertices.
    """
    triang = Delaunay(vertices[[x,y]].values)
    print('Given', len(vertices), "vertices, created", len(triang.simplices), 'triangles.')
    
    tris_df = pd.DataFrame(triang.simplices, columns=['v0', 'v1', 'v2'])
    
    return tris_df

def createHVTriMesh(x, y, var, var_name, n_workers=1):
    # Declare verts array
    # This is essentally an XYZ matrix (at the location of x and y, z=?) 3 X len(verts) array
    verts = np.column_stack([x, y, var])

    # Convert to pandas
    verts_df  = pd.DataFrame(verts,  columns=['Longitude', 'Latitude', var_name])
    
    # Creates Delaunay triangular mesh
    tris_df = triangulate(verts_df)

    # Convert to dask
    verts_ddf = dd.from_pandas(verts_df, npartitions=n_workers)
    tris_ddf = dd.from_pandas(tris_df, npartitions=n_workers)

    # Declare HoloViews element
    tri_nodes = hv.Nodes(verts_ddf, ['Longitude', 'Latitude', 'index'], [var_name])
    trimesh = hv.TriMesh((tris_ddf, tri_nodes))
    return(trimesh)

# Grid centerpoints
# if j == 0 | j == 1 | j == 2 | j == 3 | j == 4:
#     lonCell = p_mesh_ds_REF.grid_center_lon.values
#     latCell = p_mesh_ds_REF.grid_center_lat.values
# elif j == 5 | j == 7:
#     lonCell = p_mesh_ds_WAT.grid_center_lon.values
#     latCell = p_mesh_ds_WAT.grid_center_lat.values
# elif j == 6 | j == 8:
#     lonCell = p_mesh_ds_EXT.grid_center_lon.values
#     latCell = p_mesh_ds_EXT.grid_center_lat.values

lonCell = p_mesh_ds_REF.grid_center_lon.values
latCell = p_mesh_ds_REF.grid_center_lat.values
lonCell = ((lonCell - 180.0) % 360.0) - 180.0

lon_range, lat_range = basin_bboxes('florida')
#proj = ccrs.Robinson(central_longitude=0.0)
proj = ccrs.PlateCarree(central_longitude=0.0)

# Creates Delaunay triangular mesh
xPCS, yPCS, _ = proj.transform_points(ccrs.PlateCarree(), lonCell, latCell).T
x_range, y_range, _ = proj.transform_points(ccrs.PlateCarree(), np.array(lon_range), np.array(lat_range)).T
lon_range = tuple(x_range)
lat_range = tuple(y_range)

In [ ]:
# j = 0, ts = 12, 13, 18
# j = 2, ts = 12, 13
# j = 3, ts = 10, 12, 16
# j = 5, ts = 11, 17

In [ ]:
h3cn_ds[j].time.values[ts].strftime('%m%d%Y%H')

In [ ]:
h3pn_ds[j].time.values[ts]

In [ ]:
ts = 17
j = 5
times = h3cn_ds[j].time.values

primalVar = h3pn_ds[j]['FLUT'].isel(time=ts).values
trimesh = createHVTriMesh(xPCS, yPCS, primalVar, 'FLUT')
rasterized = hds_rasterize(trimesh, aggregator='mean', precompute=True, vdim_prefix='', pixel_ratio=5, x_range=lon_range, y_range=lat_range)
final = rasterized.opts(colorbar=False, cmap=flut_cimss, clim=(T_to_FLUT(-110, 'C'), T_to_FLUT(55, 'C')), width=800, height=600) *\
gf.coastline(projection=proj).options(scale='10m')
final

In [ ]:
j = 8
times = h3cn_ds[j].time.values

for i, ts in enumerate(times[2:]):
 
    primalVar = h3pn_ds[j]['FLUT'].isel(time=i).values
    trimesh = createHVTriMesh(xPCS, yPCS, primalVar, 'FLUT')
    rasterized = hds_rasterize(trimesh, aggregator='mean', precompute=True, vdim_prefix='', pixel_ratio=5, x_range=lon_range, y_range=lat_range)
    final = rasterized.opts(colorbar=False, cmap=flut_cimss, clim=(T_to_FLUT(-110, 'C'), T_to_FLUT(55, 'C')), width=800, height=600) *\
    gf.coastline(projection=proj).options(scale='10m')

    hv.save(final, f"../figs/new_tracks/{storms[j]}/CAM_{storms[j]}_{ts.strftime('%m%d%Y%H')}_nocolorbar.png", backend='bokeh', dpi=300)
    print(f'{len(times[2:])-i} figs remaining')

In [ ]:
output_folder = f"../figs/new_tracks/{storms[j]}"

from PIL import Image

# Create the frames

imgs = glob.glob(os.path.join(output_folder, "*CAM*_nocolorbar.png"))
imgs.sort()
imgs = imgs[1:]

frames = []
for i in imgs:
    new_frame = Image.open(i)
    frames.append(new_frame)
    
#Save into a GIF file that loops forever
duration = 200
frames[0].save(os.path.join(output_folder, f"CAM_{storms[j]}_FLUT_nocolorbar.gif"), format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=duration, loop=0)
new_frame.close()

# Swath Maps

## Precip

In [ ]:
# Based off of https://water.weather.gov/precip/ 30-day observed precip scale
nws_precip_colors = [
    "#ffffff",  # 0.00 - 0.01 inches  white
    "#4bd2f7",  # 0.01 - 0.10 inches  light blue
    "#699fd0",  # 0.10 - 0.25 inches  mid blue
    "#3c4bac",  # 0.25 - 0.50 inches  dark blue
    "#3cf74b",  # 0.50 - 1.00 inches  light green
    "#3cb447",  # 1.00 - 1.50 inches  mid green
    "#3c8743",  # 1.50 - 2.00 inches  dark green
    "#1f4723",  # 2.00 - 3.00 inches  darkest green
    "#f7f73c",  # 3.00 - 4.00 inches  yellow
    "#fbde88",  # 4.00 - 5.00 inches  weird tan
    "#f7ac3c",  # 5.00 - 6.00 inches  orange
    "#c47908",  # 6.00 - 8.00 inches  dark orange
    "#f73c3c",  # 8.00 - 10.00 inch  bright red
    "#bf3c3c",  # 10.00 - 15.00 inch  mid red
    "#6e2b2b",  # 15.00 - 20.00 inch  dark red
    "#f73cf7",  # 20.00 - 25.00 inch  bright pink
    "#9974e4",  # 25.00 - 30.00 inch  purple
    #"#404040",  # 30.00 - 40.00 inch  dark gray because of mpl
    "#c2c2c2"  # 30.00 - 40.00 inch  gray
    ]
nws_cmap = mpl.colors.ListedColormap(nws_precip_colors, 'nws_precip')
levels = [0.0, 0.01, 0.10, 0.25, 0.50, 1.0, 1.5, 2.0, 3.0, 4.0, 5.0, 6.0, 8.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0]
#levels = [0.01, 0.10, 0.25, 0.50, 1.0, 1.5, 2.0, 3.0, 4.0, 5.0, 6.0, 8.0, 10.0, 15.0, 20.0, 25.0, 30.0, 40.0]
nws_norm = mpl.colors.BoundaryNorm(boundaries=levels, ncolors=len(levels))
#mpl.colormaps.register(nws_cmap, force=True)
nws_cmap

In [ ]:
for i, ds in enumerate(h4pr_ds):
    ds['tot_prect'] = ds['PRECT'].metpy.convert_units('inch/hour').sum(dim='time') * units('hour')
    ds['max_hrly_prect'] = ds['PRECT'].metpy.convert_units('inch/hour').max(dim='time') * units('hour')
    
    h4pn_ds[i]['tot_prect'] = h4pn_ds[i]['PRECT'].metpy.convert_units('inch/hour').sum(dim='time') * units('hour')
    h4pn_ds[i]['max_hrly_prect'] = h4pn_ds[i]['PRECT'].metpy.convert_units('inch/hour').max(dim='time') * units('hour')

    h4cr_ds[i]['tot_prect'] = h4cr_ds[i]['PRECT'].metpy.convert_units('inch/hour').sum(dim='time') * units('hour')
    h4cr_ds[i]['max_hrly_prect'] = h4cr_ds[i]['PRECT'].metpy.convert_units('inch/hour').max(dim='time') * units('hour')

    h4cn_ds[i]['tot_prect'] = h4cn_ds[i]['PRECT'].metpy.convert_units('inch/hour').sum(dim='time') * units('hour')
    h4cn_ds[i]['max_hrly_prect'] = h4cn_ds[i]['PRECT'].metpy.convert_units('inch/hour').max(dim='time') * units('hour')

In [ ]:
# https://discourse.holoviz.org/t/custom-discrete-colormaps/2183/2
# Convert this using hooks

# import pandas as pd
# import numpy as np
# import holoviews as hv
# from holoviews import opts
# from bokeh.palettes import all_palettes
# from bokeh.models import FuncTickFormatter
# from bokeh.plotting import show

df = pd.DataFrame(columns=['x','y','val','bin'])
df['x'] = list(range(10))
df['y'] = list(range(10))
df['val'] = np.random.randint(110,size=10)
df['bin'] = (df.val/10).astype(int)
plot = hv.Scatter(df,kdims=['x'],vdims=['y','bin']).opts(color='bin',colorbar=True,tools=['hover'],color_levels=11)
plot

def custom_colorbar(plot,palette,labels=None):
    plot = plot.opts(color_levels=10)
    min_ = 0
    nb_color = len(palette)
    max_=2*nb_color
    rend = hv.render(plot)
    cmapper = rend.right[0].color_mapper
    ticker = rend.right[0].ticker
    cmapper.high = nb_color
    cmapper.palette = palette
    ticker.desired_num_ticks = nb_color+1
    ticker.num_minor_ticks = 0
    step = int((max_-min_)/nb_color)
    ticker.base=2
    ticker.mantissas = list(range(min_,max_+step,step))
    if labels is not None and len(labels) == nb_color+1:
        Xlabel = dict(zip(list(range(nb_color+1)),labels))
        formatter = FuncTickFormatter(code="""
                    var labels = %s;
                    return labels[tick] || tick;
                    """ % Xlabel)
        rend.right[0].formatter = formatter
    return rend

labels = ['0', '200', '400', '600','800', '1000', '2000','3000','4000','5000','6000']
new_rend = custom_colorbar(plot,all_palettes['BrBG'][10],labels)
show(new_rend)

In [ ]:
def custom_colorbar(plot,palette,labels=None):
    plot = plot.opts(color_levels=10)
    min_ = 0
    nb_color = len(palette)
    max_=2*nb_color
    rend = hv.render(plot)
    cmapper = rend.right[0].color_mapper
    ticker = rend.right[0].ticker
    cmapper.high = nb_color
    cmapper.palette = palette
    ticker.desired_num_ticks = nb_color+1
    ticker.num_minor_ticks = 0
    step = int((max_-min_)/nb_color)
    ticker.base=2
    ticker.mantissas = list(range(min_,max_+step,step))
    if labels is not None and len(labels) == nb_color+1:
        Xlabel = dict(zip(list(range(nb_color+1)),labels))
        formatter = FuncTickFormatter(code="""
                    var labels = %s;
                    return labels[tick] || tick;
                    """ % Xlabel)
        rend.right[0].formatter = formatter
    return rend

labels = ['0', '200', '400', '600','800', '1000', '2000','3000','4000','5000','6000']
new_rend = custom_colorbar(plot,all_palettes['BrBG'][10],labels)
show(new_rend)

In [ ]:
major_label_overrides=dict(list(zip(levels, list(map(str, levels))))), 

mpas_total = rasterized.opts(tools=['hover'], colorbar=True, cmap=nws_cmap, title='MPAS (Native) Total Precipitation', 
                                    clim=(levels[0], levels[-1]), width=1000, height=800, color_levels=levels, 
                                    clipping_colors={'NaN':'#FFFFFF'}, clabel='Accumulated Precipitation [inches]', colorbar_position='bottom', 
                                    fontsize=dict(title='30pt', clabel='28pt', cticks='26pt'), alpha=0.75) *\
gf.coastline(projection=proj, line_width=2.0, alpha=1).options(scale='10m') *\
gf.lakes(projection=proj, line_width=2.0, fill_color='none', line_color='#323232', alpha=1).options(scale='10m')

In [ ]:
cbar_levs= [0.0, 0.01, 0.10, 0.25, 0.50, 1.0, 1.5, 2.0, 3.0, 4.0]
cbar_cols = nws_precip_colors[:9]
print(len(cbar_levs))
print(len(cbar_cols))

In [ ]:
print(len(levels))
print(len(nws_precip_colors))

In [ ]:
cbar_cols

In [ ]:
j = 0
w = 2000 #1600
h = 1600 #1200

#clipping_colors={'NaN':(0, 0, 0, 0)}, #sets NaN to be transparent, rather than white (last 0 is alpha)  *\
#cbar_labs = {0:'0.0', 1:'1.0', 2:'2.0', 3:'3.0', 4:'4.0', 5:'5.0', 6:'6.0', 8:'8.0', 10:'10.0', 15:'15.0', 20:'20.0', 25:'25.0', 30:'30.0', 40:'40.0'}
#cbar_levs = [0, 2, 4, 6, 8, 10, 15, 20, 25, 30, 35]

cbar_levs= [0.0, 0.01, 0.10, 0.25, 0.50, 1.0, 1.5, 2.0, 3.0, 4.0]
cbar_cols = nws_precip_colors[:9]
datashader_opts=dict(aggregator='mean', precompute=True, vdim_prefix='', pixel_ratio=5, x_range=lon_range, y_range=lat_range)
plot_opts_c = dict(tools=['hover'], cmap=cbar_cols, xaxis=None, yaxis=None, color_levels=cbar_levs, width=w, height=h, 
                 colorbar=True, colorbar_position='bottom', clabel='Maximum Precipitation [inches]', alpha=0.75, clim=(0.0, 4.0), 
                   clipping_colors={'NaN':'#FFFFFF'},
                 fontsize=dict(title='70pt', clabel='54pt', cticks='50pt'), title='CAM5-MPAS Maximum Hourly Precipitation', 
                 colorbar_opts=dict(height=50, scale_alpha=0.75, border_line_width=3, title_text_font='arial', 
                                    ticker=FixedTicker(ticks=cbar_levs), major_label_text_font='arial')) # title_text_font_style='normal',
coastline_opts = dict(scale='10m', line_color='#000000', line_width=3.0, width=w, height=h)
lakes_opts = dict(projection=proj, line_width=3.0, line_color='#000000', fill_color='none')

var = 'max_hrly_prect'
primalVar_c = h4cn_ds[j][var].values
#primalVar_c = h4cn_ds[j][var].where(h4cn_ds[j][var].values >= 0.0099).values
#primalVar_c = h4cn_ds[j][var].where(h4cn_ds[j][var].values >= 0.01).values
trimesh_c = createHVTriMesh(xPCS_c, yPCS_c, primalVar_c, var, tris_ccw_flat, n_workers=1)
rasterized_c = hds_rasterize(trimesh_c, **datashader_opts)
mpas_mh_precip = rasterized_c.opts(**plot_opts_c) * gf.coastline(projection=proj).opts(**coastline_opts) * gf.lakes(**lakes_opts).opts(scale='10m')

In [ ]:
w = 2000 #1600
h = 1400 #1100

datashader_opts=dict(aggregator='mean', precompute=True, vdim_prefix='', pixel_ratio=5, x_range=lon_range, y_range=lat_range)
plot_opts_p = dict(tools=['hover'], colorbar=False, cmap=cbar_cols, xaxis=None, yaxis=None, color_levels=cbar_levs, width=w, height=h, clim=(0.0, 4.0),
                 alpha=0.75, fontsize=dict(title='70pt'), title='CAM5-SE Maximum Hourly Precipitation')
coastline_opts = dict(scale='10m', line_color='#000000', line_width=3.0, width=w, height=h)
lakes_opts = dict(projection=proj, line_width=3.0, line_color='#000000')

var = 'max_hrly_prect'
primalVar_p = h4pn_ds[j][var].values
trimesh_p = createHVTriMesh(xPCS_p, yPCS_p, primalVar_p, var)
rasterized_p = hds_rasterize(trimesh_p, **datashader_opts)
cam_mh_precip = rasterized_p.opts(**plot_opts_p) * gf.coastline(projection=proj).opts(**coastline_opts) * gf.lakes(**lakes_opts).opts(scale='10m') 
#cam_mh_precip

In [ ]:
precip_figure = hv.Layout([cam_mh_precip, mpas_mh_precip]).cols(1)
hv.save(precip_figure, '../figs/storm_0236_max_hrly_prect2.png', dpi=300, center=False)

In [ ]:
levels[-1]

In [ ]:
j = 0
w = 2000 #1600
h = 1600 #1200

#clipping_colors={'NaN':(0, 0, 0, 0)}, #sets NaN to be transparent, rather than white (last 0 is alpha)  *\
#cbar_labs = {0:'0.0', 1:'1.0', 2:'2.0', 3:'3.0', 4:'4.0', 5:'5.0', 6:'6.0', 8:'8.0', 10:'10.0', 15:'15.0', 20:'20.0', 25:'25.0', 30:'30.0', 40:'40.0'}
cbar_levs = [0, 2, 4, 6, 8, 10, 15, 20, 25, 30, 35]
datashader_opts=dict(aggregator='mean', precompute=True, vdim_prefix='', pixel_ratio=5, x_range=lon_range, y_range=lat_range)
plot_opts_c = dict(tools=['hover'], cmap=nws_precip_colors, xaxis=None, yaxis=None, color_levels=levels, width=w, height=h, 
                 colorbar=True, colorbar_position='bottom', clabel='Accumulated Precipitation [inches]', alpha=0.75, clim=(0.0, 35.0), 
                 fontsize=dict(title='75pt', clabel='54pt', cticks='50pt'), title='CAM5-MPAS Total Precipitation', 
                 colorbar_opts=dict(height=50, scale_alpha=0.75, border_line_width=3, ticker=FixedTicker(ticks=cbar_levs), title_text_font='arial', major_label_text_font='arial')) # title_text_font_style='normal',
coastline_opts = dict(scale='10m', line_color='#000000', line_width=3.0, width=w, height=h)
lakes_opts = dict(projection=proj, line_width=3.0, line_color='#000000', fill_color='none')

var = 'tot_prect'
#primalVar_c = h4cn_ds[j][var].values
#primalVar_c = h4cn_ds[j][var].where(h4cn_ds[j][var].values >= 0.0099).values
primalVar_c = h4cn_ds[j][var].where(h4cn_ds[j][var].values >= 0.01).values
trimesh_c = createHVTriMesh(xPCS_c, yPCS_c, primalVar_c, var, tris_ccw_flat, n_workers=1)
rasterized_c = hds_rasterize(trimesh_c, **datashader_opts)
mpas_precip = rasterized_c.opts(**plot_opts_c) * gf.coastline(projection=proj).opts(**coastline_opts) * gf.lakes(**lakes_opts).opts(scale='10m')

#mpas_precip

In [ ]:
j = 0

w = 2000 #1600
h = 1400 #1100

datashader_opts=dict(aggregator='mean', precompute=True, vdim_prefix='', pixel_ratio=5, x_range=lon_range, y_range=lat_range)
plot_opts_p = dict(colorbar=False, cmap=nws_cmap, xaxis=None, yaxis=None, color_levels=levels, width=w, height=h, clipping_colors={'NaN':(0, 0, 0, 0)},
                 alpha=0.75, fontsize=dict(title='75pt'), title='CAM5-SE Total Precipitation')
coastline_opts = dict(scale='10m', line_color='#000000', line_width=3.0, width=w, height=h)
lakes_opts = dict(projection=proj, line_width=3.0, line_color='#000000', fill_color='none')

var = 'tot_prect'
primalVar_p = h4pn_ds[j][var].where(h4pn_ds[j][var].values >= 0.01).values
trimesh_p = createHVTriMesh(xPCS_p, yPCS_p, primalVar_p, var)
rasterized_p = hds_rasterize(trimesh_p, **datashader_opts)
cam_precip = rasterized_p.opts(**plot_opts_p) * gf.coastline(projection=proj).opts(**coastline_opts) * gf.lakes(**lakes_opts).opts(scale='10m') 
#cam_precip

In [ ]:
precip_figure = hv.Layout([cam_precip, mpas_precip]).cols(1)
hv.save(precip_figure, '../figs/storm_0236_total_prect2.png', dpi=300, center=False)

In [ ]:
proj=ccrs.PlateCarree()
    
fig, ax = plt.subplots(figsize=(7,6), dpi=300, subplot_kw=dict(projection=proj), layout='constrained')

ts = 18
alpha = 0.75

# How I normally make my plots
#tot_precip_masked = h4cr_ds[0]['tot_prect'].where(h4cr_ds[0]['tot_prect'] >= 0.0).fillna(0)
tot_precip_masked.plot(ax=ax, norm=nws_norm, cmap=nws_cmap, levels=levels, vmax=40.0, add_colorbar=True, alpha=alpha)
#h4cr_ds[0]['tot_prect'].plot(ax=ax, vmin=0.0, cmap=nws_cmap, add_colorbar=True, alpha=alpha)

# Adding levels, vmin, and vmax flags (should be included in norm flag but it makes no difference anyway)
#h4cr_ds[0]['tot_prect'].plot(ax=ax, norm=nws_norm, levels=levels, vmin=levels[0], vmax=levels[-1], cmap=nws_cmap, add_colorbar=True, alpha=alpha)
ax.set_title('MPAS (Regridded) Total Precipitation', fontsize=ts, fontweight='bold')

ax.set_extent([lons[0], lons[1], lats[0], lats[1]])
ax.coastlines(resolution='10m', linewidth=1, edgecolor='#323232')
ax.add_feature(cfeature.LAKES.with_scale('10m'), linewidth=1, facecolor='none', edgecolor='#323232', zorder=2)
plt.show()

# cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=nws_norm, cmap=nws_cmap), orientation='horizontal', ax=ax, ticks=levels, alpha=alpha)
# cbar.ax.tick_params(labelsize=14)
# cbar.set_label(label='Accumulated Precipitation [inches]', size=16, fontstyle='italic')

In [ ]:
# Repeats basin bbox by number of storms for easier mapping to functions
bboxes = [basin_bboxes('florida')]*len(storms)
list(map(plot_regridded_precip_swaths, h4pr_ds, h4cr_ds, storms, bboxes, [False]*len(storms))) 

In [ ]:
# Repeats basin bbox by number of storms for easier mapping to functions
bboxes = [basin_bboxes('florida')]*len(storms)
list(map(plot_regridded_precip_swaths, h4pr_ds, h4cr_ds, storms, bboxes, [True]*len(storms))) # For South Florida subset

## Wind

In [ ]:
h3pn_ds[0]

In [ ]:
for i, ds in enumerate(h3pr_ds):
    ds['max_sfc_wsp'] = ds['U10'].max(dim='time')
    ds['max_850_wsp'] = ds['WSP850'].max(dim='time')
    
    h3pn_ds[i]['max_sfc_wsp'] = h3pn_ds[i]['U10'].max(dim='time')
    h3pn_ds[i]['max_850_wsp'] = h3pn_ds[i]['WSP850'].max(dim='time')
    
    h3cn_ds[i]['max_sfc_wsp'] = h3cn_ds[i]['U10'].max(dim='time')
    h3cn_ds[i]['max_850_wsp'] = h3cn_ds[i]['WSP850'].max(dim='time')
    
    h3cr_ds[i]['max_sfc_wsp'] = h3cr_ds[i]['U10'].max(dim='time')
    h3cr_ds[i]['max_850_wsp'] = h3cr_ds[i]['WSP850'].max(dim='time')

In [ ]:
import bokeh.palettes
viridis_colors = bokeh.palettes.viridis(256)
viridis_reversed = viridis_colors[::-1]
viridis_1 = viridis_colors[:128]
viridis_2 = viridis_colors[128:]

In [ ]:
lons, lats = basin_bboxes('florida')

ncolors = 256
#cmin = h3cr_ds[0]['max_sfc_wsp'].sel(lon=slice(lons[0], lons[1]), lat=slice(lats[0], lats[1])).values.min()
#cmax = h3cr_ds[0]['max_sfc_wsp'].sel(lon=slice(lons[0], lons[1]), lat=slice(lats[0], lats[1])).values.max()
cmin = 0
cmax = 130
cmid = 74.0
c_diverge_point_normalized = (cmid - cmin) / (cmax - cmin)
palette_cutoff = round(c_diverge_point_normalized * ncolors)
v2 = viridis_colors[palette_cutoff:]
wsp_colors = bokeh.palettes.diverging_palette(viridis_colors[:palette_cutoff], v2[::-1], n=256, midpoint=c_diverge_point_normalized)

# c_diverge_point_normalized = (cmid - cmin) / (cmax - cmin)
# palette_cutoff = round(c_diverge_point_normalized * ncolors)
# colors = bokeh.palettes.diverging_palette(
#     bokeh.palettes.Reds[ncolors],
#     bokeh.palettes.Blues[ncolors][palette_cutoff:],
#     n=ncolors,
#     midpoint=c_diverge_point_normalized,
# )

In [ ]:
j = 0
w = 2000 #1600
h = 1600 #1200

cbar_levs = [0, 20, 40, 60, 80, 100, 130]
datashader_opts=dict(aggregator='mean', precompute=True, vdim_prefix='', pixel_ratio=5, x_range=lon_range, y_range=lat_range)
plot_opts_c = dict(cmap=wsp_colors, xaxis=None, yaxis=None, clim=(cmin, cmax), width=w, height=h, 
                 colorbar=True, colorbar_position='bottom', clabel='Maximum Wind Speed [mph]', alpha=0.75,
                 fontsize=dict(title='70pt', clabel='54pt', cticks='50pt'), title='CAM5-MPAS Maximum Surface Wind Speed', 
                 colorbar_opts=dict(height=50, scale_alpha=0.75, border_line_width=3, ticker=FixedTicker(ticks=cbar_levs), 
                                    title_text_font='arial', major_label_text_font='arial')) # title_text_font_style='normal',
coastline_opts = dict(scale='10m', line_color='#000000', line_width=3.0, width=w, height=h)
lakes_opts = dict(projection=proj, line_width=3.0, line_color='#000000', fill_color='none')

var = 'max_sfc_wsp'
primalVar_c = h3cn_ds[j][var].values
trimesh_c = createHVTriMesh(xPCS_c, yPCS_c, primalVar_c, var, tris_ccw_flat, n_workers=1)
rasterized_c = hds_rasterize(trimesh_c, **datashader_opts)
mpas_wind = rasterized_c.opts(**plot_opts_c) * gf.coastline(projection=proj).opts(**coastline_opts) * gf.lakes(**lakes_opts).opts(scale='10m')

#mpas_wind

In [ ]:
print(cmin)
print(cmax)

In [ ]:
j = 0

w = 2000 #1600
h = 1400 #1100

datashader_opts=dict(aggregator='mean', precompute=True, vdim_prefix='', pixel_ratio=5, x_range=lon_range, y_range=lat_range)
plot_opts_p = dict(colorbar=False, cmap=wsp_colors, xaxis=None, yaxis=None, width=w, height=h, clim=(cmin, cmax),
                 alpha=0.75, fontsize=dict(title='70pt'), title='CAM5-SE Maximum Surface Wind Speed')
coastline_opts = dict(scale='10m', line_color='#000000', line_width=3.0, width=w, height=h)
lakes_opts = dict(projection=proj, line_width=3.0, line_color='#000000', fill_color='none')

var = 'max_sfc_wsp'
primalVar_p = h3pn_ds[j][var].values
trimesh_p = createHVTriMesh(xPCS_p, yPCS_p, primalVar_p, var)
rasterized_p = hds_rasterize(trimesh_p, **datashader_opts)
cam_wind = rasterized_p.opts(**plot_opts_p) * gf.coastline(projection=proj).opts(**coastline_opts) * gf.lakes(**lakes_opts).opts(scale='10m') 

In [ ]:
wind_figure = hv.Layout([cam_wind, mpas_wind]).cols(1)
hv.save(wind_figure, '../figs/storm_0236_max_sfc_wsp2.png', dpi=300, center=False)

In [ ]:
lons, lats = basin_bboxes('florida')
proj=ccrs.PlateCarree()

fig, axs = plt.subplots(1, 2, figsize=(14,8), dpi=300, subplot_kw=dict(projection=proj), layout='constrained')

ts = 18
alpha = 0.75

vmin1 = h3cr_ds[0]['max_sfc_wsp'].sel(lon=slice(lons[0], lons[1]), lat=slice(lats[0], lats[1])).values.min()
vmax1 = h3cr_ds[0]['max_sfc_wsp'].sel(lon=slice(lons[0], lons[1]), lat=slice(lats[0], lats[1])).values.max()
if vmax1 > 74:
    norm1 = mcolors.TwoSlopeNorm(vmin=vmin1, vcenter=74, vmax=vmax1)
else:
    norm1 = mcolors.TwoSlopeNorm(vmin=vmin1, vcenter=38, vmax=vmax1)

h3pr_ds[0]['max_sfc_wsp'].plot(ax=axs[0], norm=norm1, cmap='viridis', add_colorbar=False, alpha=alpha)
axs[0].set_title('CAM5 (Regridded) Maximum Surface WSP', fontsize=ts, fontweight='bold')

h3cr_ds[0]['max_sfc_wsp'].plot(ax=axs[1], norm=norm1, cmap='viridis', add_colorbar=False, alpha=alpha)
axs[1].set_title('MPAS (Regridded) Maximum Surface WSP', fontsize=ts, fontweight='bold')

for ax in axs.ravel():
    ax.set_extent([lons[0], lons[1], lats[0], lats[1]])
    ax.coastlines(resolution='10m', linewidth=1, edgecolor='#323232')
    ax.add_feature(cfeature.LAKES.with_scale('10m'), linewidth=1, facecolor='none', edgecolor='#323232', zorder=2)

cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm1, cmap='viridis'), orientation='horizontal', ax=axs)
cbar.ax.tick_params(labelsize=14)
cbar.set_label(label='Wind Speed [mph]', size=16, fontstyle='italic')

In [ ]:
# Repeats basin bbox by number of storms for easier mapping to functions
bboxes = [basin_bboxes('florida')]*len(storms)
list(map(plot_regridded_wind_swaths, h3pr_ds, h3cr_ds, storms, bboxes, [False]*len(storms)) 

In [ ]:
# Repeats basin bbox by number of storms for easier mapping to functions
bboxes = [basin_bboxes('florida')]*len(storms)
list(map(plot_regridded_wind_swaths, h3pr_ds, h3cr_ds, storms, bboxes, [True]*len(storms))) 

# Precipitation Histograms

In [ ]:
# FIX THIS HOT MESS
def precip_metrics(ds, var, bbox):
    
    subset_ds = get_subset(ds, bbox)
    
    # # Slices by lat/lon bounding box
    # if 'lon' in ds.dims:
    #     ds = ds.sel(lon=slice(int(bbox[0][0]), int(bbox[0][1])), lat=slice(int(bbox[1][0]), int(bbox[1][1])))
    # else:
    #     try:
    #         ds['lon'] = ds['lon'].load()
    #         ds['lat'] = ds['lat'].load()
    #         ds = ds.where((ds['lon'] >= bbox[0][0]) & (ds['lon'] <= bbox[0][1]) &
    #                       (ds['lat'] >= bbox[1][0]) & (ds['lat'] <= bbox[1][1]), drop=True)
    #     except NotImplementedError:
    #         ds = ds.where((ds['lon'] >= bbox[0][0]) & (ds['lon'] <= bbox[0][1]) &
    #                       (ds['lat'] >= bbox[1][0]) & (ds['lat'] <= bbox[1][1]), drop=True)
        
    # Filter out > 0.01 mm rain
    
    try:
        tot_precip = subset_ds[var].metpy.convert_units('inch/hour').sum(dim='time') * units('hour')
        max_precip = subset_ds[var].metpy.convert_units('inch/hour').max(dim='time') * units('hour')
    except DimensionalityError:
        subset_ds[var] = subset_ds[var] * units('m/s')
        tot_precip = subset_ds[var].metpy.convert_units('inch/hour').sum(dim='time') * units('hour')
        max_precip = subset_ds[var].metpy.convert_units('inch/hour').max(dim='time') * units('hour')
    
    tot_precip = tot_precip.where(tot_precip >= 0.0).fillna(0)
    max_precip = max_precip.where(max_precip >= 0.0).fillna(0)
    return tot_precip, max_precip

In [ ]:
def precip_plots(nparent_ds, rparent_ds, nchild_ds, rchild_ds, storm_name, bbox, stat='density', subset=False, savefig=True):
    
    # Total sum (inches) and max hourly (inches)
    cn_tp, cn_mp = precip_metrics(nchild_ds, 'PRECT', bbox)
    cr_tp, cr_mp = precip_metrics(rchild_ds, 'PRECT', bbox)
    pn_tp, pn_mp = precip_metrics(nparent_ds, 'PRECT', bbox)
    pr_tp, pr_mp = precip_metrics(rparent_ds, 'PRECT', bbox)
    
    lw=1.25
    
    print(f"Plotting {storm_name}.")
    fig, axs = plt.subplots(2, 2, figsize=(15, 15), dpi=300, layout='constrained')
    sns.histplot(pn_tp.values.flatten(), stat=stat, element='step', color='#024588', alpha=0.0005, label='CAM (Native)', 
                 kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[0][0])
    sns.histplot(cn_tp.values.flatten(), stat=stat, element='step', color='#FA03B2', alpha=0.0005, label='MPAS (Native)', 
                 kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[0][0])
    axs[0][0].set_title('Total Precipitation', fontsize=22, fontweight='bold')
    #axs[0][0].tick_params(labelbottom=False)
    axs[0][0].set_ylabel('Density', fontsize=18)

    
    g3 = sns.histplot(pn_mp.values.flatten(), stat=stat, element='step', color='#024588', alpha=0.0005, label='CAM (Native)', 
                 kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[0][1])
    g4 = sns.histplot(cn_mp.values.flatten(), stat=stat, element='step', color='#FA03B2', alpha=0.0005, label='MPAS (Native)', 
                 kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[0][1])
    axs[0][1].set_title('Maximum Hourly Precipitation', fontsize=22, fontweight='bold')
    #axs[0][1].tick_params(labelbottom=False)
    axs[0][1].set_ylabel(None)

    sns.histplot(pr_tp.values.flatten(), stat=stat, element='step', color='#7a8fc2', alpha=0.0005, label='CAM (Regridded)', 
                 kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[1][0])
    sns.histplot(cr_tp.values.flatten(), stat=stat, element='step', color='#ff8bd1', alpha=0.0005, label='MPAS (Regridded)', 
                 kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[1][0])
    axs[1][0].set_xlabel('Accumulated Precipitation [inches]', fontsize=18)
    axs[1][0].set_ylabel('Density', fontsize=18)

    sns.histplot(pr_mp.values.flatten(), stat=stat, element='step', color='#7a8fc2', alpha=0.0005, label='CAM (Regridded)', 
                 kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[1][1])
    sns.histplot(cr_mp.values.flatten(), stat=stat, element='step', color='#ff8bd1', alpha=0.0005, label='MPAS (Regridded)', 
                 kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[1][1])
    axs[1][1].set_xlabel('Accumulated Precipitation [inches]', fontsize=18)
    axs[1][1].set_ylabel(None)
    
    for ax in axs.ravel():
        ax.legend(fontsize=18)
        ax.tick_params(axis='both', which='major', labelsize=16)
    
    if savefig == True:
        if subset == False:
            fig.savefig(f"../figs/new_tracks/{storm_name}/{storm_name}_precip_histograms.png", bbox_inches='tight', dpi=300)
        else:
            fig.savefig(f"../figs/new_tracks/{storm_name}/{storm_name}_precip_histograms_south_florida.png", bbox_inches='tight', dpi=300)
    
    plt.close()

In [ ]:
bbox

In [ ]:
cn_tp.values.max()

In [ ]:
cn_subset = get_subset(h4cn_ds[0], basin_bboxes('florida'))
cn_subset['tot_prect'].values.max()

In [ ]:
j = 0
nchild_ds = h3cn_ds[j]
#rchild_ds = h3cr_ds[j]
nparent_ds = h3pn_ds[j]
#rparent_ds = h3pr_ds[j]

#bbox = basin_bboxes('south florida')
west = -84.0+360.0
east = -79.0+360
bbox = ((west, east), (24.0, 30.0))
#bbox = basin_bboxes('florida')
# Total sum (inches) and max hourly (inches)
cn_subset = get_subset(h4cn_ds[0], bbox)
cn_tp = cn_subset['tot_prect']
cn_mp = cn_subset['max_hrly_prect']

pn_subset = get_subset(h4pn_ds[0], bbox)
pn_tp = pn_subset['tot_prect']
pn_mp = pn_subset['max_hrly_prect']

#cn_tp, cn_mp = precip_metrics(nchild_ds, 'PRECT', bbox)
#cr_tp, cr_mp = precip_metrics(rchild_ds, 'PRECT', bbox)
#pn_tp, pn_mp = precip_metrics(nparent_ds, 'PRECT', bbox)
#pr_tp, pr_mp = precip_metrics(rparent_ds, 'PRECT', bbox)

lw=2.0
titles = 32 #28
labels = 24 #18
ticks = 20 #16
legends = 26 #20

#print(f"Plotting {storm_name}.")  #e892c4
stat='density'
fig, axs = plt.subplots(1, 2, figsize=(20, 6.5), dpi=300, layout='constrained') # rows, cols, w, h
sns.histplot(pn_tp.values.flatten(), stat=stat, element='step', color='#024588', alpha=0.0005, label='CAM5-SE', 
             kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[1])
sns.histplot(cn_tp.values.flatten(), stat=stat, element='step', color='#BD0086', alpha=0.0005, label='CAM5-MPAS', 
             kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[1])
axs[1].set_title('Total Precipitation', fontsize=titles, fontweight='bold', loc='left')
axs[1].set_xlabel('Accumulated Precipitation [inches]', fontsize=labels)
axs[1].set_ylabel(None)


g3 = sns.histplot(pn_mp.values.flatten(), stat=stat, element='step', color='#024588', alpha=0.0005, label='CAM5-SE', 
             kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[0])
g4 = sns.histplot(cn_mp.values.flatten(), stat=stat, element='step', color='#BD0086', alpha=0.0005, label='CAM5-MPAS', 
             kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[0])
axs[0].set_title('Maximum Hourly Precipitation', fontsize=titles, fontweight='bold', loc='left')
axs[0].set_xlabel('Maximum Precipitation [inches]', fontsize=labels)
axs[0].set_ylabel('Density', fontsize=labels)


for ax in axs.ravel():
    ax.legend(fontsize=legends)
    ax.tick_params(axis='both', which='major', labelsize=ticks)

fig.savefig(f"../figs/{storms[j]}_precip_histograms2.png", bbox_inches='tight', dpi=300)
plt.close()
#plt.show()

In [ ]:
# Repeats basin bbox by number of storms for easier mapping to functions
bboxes = [basin_bboxes('florida')]*len(storms)
list(map(precip_plots, h4pn_ds, h4pr_ds, h4cn_ds, h4cr_ds, storms, bboxes, ['density']*len(storms), [False]*len(storms)))

In [ ]:
# Repeats basin bbox by number of storms for easier mapping to functions
bboxes2 = [basin_bboxes('south florida')]*len(storms)
list(map(precip_plots, h4pn_ds, h4pr_ds, h4cn_ds, h4cr_ds, storms, bboxes2, ['density']*len(storms), [True]*len(storms)))

# Wind Speed Histograms

In [ ]:
def wsp_plots(nparent_ds, rparent_ds, nchild_ds, rchild_ds, storm_name, bbox, stat='density', subset=False, savefig=True):
    
    lw=1.25
    
    if subset == True:
        lons_sfl, lats_sfl = basin_bboxes('south florida')
        rparent_ds = rparent_ds.sel(lon=slice(lons_sfl[0], lons_sfl[1]), lat=slice(lats_sfl[0], lats_sfl[1]))
        rchild_ds = rchild_ds.sel(lon=slice(lons_sfl[0], lons_sfl[1]), lat=slice(lats_sfl[0], lats_sfl[1]))
        
        nparent_ds = nparent_ds.where((nparent_ds['lon'] >=lons_sfl[0]) & (nparent_ds['lon'] <= lons_sfl[1]) &
              (nparent_ds['lat'] >= lats_sfl[0]) & (nparent_ds['lat'] <= lats_sfl[1]), drop=True)
        nchild_ds = nchild_ds.where((nchild_ds['lon'] >=lons_sfl[0]) & (nchild_ds['lon'] <= lons_sfl[1]) &
              (nchild_ds['lat'] >= lats_sfl[0]) & (nchild_ds['lat'] <= lats_sfl[1]), drop=True)
        
    print(f"Plotting {storm_name}.")
    fig, axs = plt.subplots(2, 2, figsize=(15, 15), dpi=300, layout='constrained')
    sns.histplot(nparent_ds['max_sfc_wsp'].values.flatten(), stat=stat, element='step', color='#024588', alpha=0.0005, label='CAM (Native)', 
                 kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[0][0])
    sns.histplot(nchild_ds['max_sfc_wsp'].values.flatten(), stat=stat, element='step', color='#BD0086', alpha=0.0005, label='MPAS (Native)', 
                 kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[0][0])
    axs[0][0].set_title('Maximum Surface Wind Speed', fontsize=22, fontweight='bold')
    axs[0][0].set_ylabel('Density', fontsize=18)
    
    g3 = sns.histplot(nparent_ds['max_850_wsp'].values.flatten(), stat=stat, element='step', color='#024588', alpha=0.0005, label='CAM (Native)', 
                 kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[0][1])
    g4 = sns.histplot(nchild_ds['max_850_wsp'].values.flatten(), stat=stat, element='step', color='#BD0086', alpha=0.0005, label='MPAS (Native)', 
                 kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[0][1])
    axs[0][1].set_title('Maximum 850 hPa Wind Speed', fontsize=22, fontweight='bold')
    axs[0][1].set_ylabel(None)

    sns.histplot(rparent_ds['max_sfc_wsp'].values.flatten(), stat=stat, element='step', color='#7a8fc2', alpha=0.0005, label='CAM (Regridded)', 
                 kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[1][0])
    sns.histplot(rchild_ds['max_sfc_wsp'].values.flatten(), stat=stat, element='step', color='#ff8bd1', alpha=0.0005, label='MPAS (Regridded)', 
                 kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[1][0])
    axs[1][0].set_xlabel('Wind Speed [mph]', fontsize=18)
    axs[1][0].set_ylabel('Density', fontsize=18)

    sns.histplot(rparent_ds['max_850_wsp'].values.flatten(), stat=stat, element='step', color='#7a8fc2', alpha=0.0005, label='CAM (Regridded)', 
                 kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[1][1])
    sns.histplot(rchild_ds['max_850_wsp'].values.flatten(), stat=stat, element='step', color='#ff8bd1', alpha=0.0005, label='MPAS (Regridded)', 
                 kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[1][1])
    axs[1][1].set_xlabel('Wind Speed [mph]', fontsize=18)
    axs[1][1].set_ylabel(None)
    
    for ax in axs.ravel():
        ax.legend(fontsize=18)
        ax.tick_params(axis='both', which='major', labelsize=16)
    
    if savefig == True:
        if subset == False:
            fig.savefig(f"../figs/new_tracks/{storm_name}/{storm_name}_wsp_histograms.png", bbox_inches='tight', dpi=300)
        else:
            fig.savefig(f"../figs/new_tracks/{storm_name}/{storm_name}_wsp_histograms_south_florida.png", bbox_inches='tight', dpi=300)
    
    plt.close()

In [ ]:
# Repeats basin bbox by number of storms for easier mapping to functions
bboxes = [basin_bboxes('florida')]*len(storms)
list(map(wsp_plots, h3pn_ds, h3pr_ds, h3cn_ds, h3cr_ds, storms, bboxes, ['density']*len(storms), [False]*len(storms)))

In [ ]:
# Repeats basin bbox by number of storms for easier mapping to functions
bboxes2 = [basin_bboxes('south florida')]*len(storms)
list(map(wsp_plots, h3pn_ds, h3pr_ds, h3cn_ds, h3cr_ds, storms, bboxes2, ['density']*len(storms), [True]*len(storms))) # For south florida plots

In [ ]:
j = 0
nchild_ds = h3cn_ds[j]
#rchild_ds = h3cr_ds[j]
nparent_ds = h3pn_ds[j]
#rparent_ds = h3pr_ds[j]

#bbox = basin_bboxes('south florida')
west = -84.0+360.0
east = -79.0+360
bbox = ((west, east), (24.0, 30.0))
#bbox = basin_bboxes('florida')
# Total sum (inches) and max hourly (inches)
cn_subset = get_subset(h3cn_ds[0], bbox)
pn_subset = get_subset(h3pn_ds[0], bbox)

lw=2.0
titles = 32 #28
labels = 24 #18
ticks = 20 #16
legends = 26 #20

#print(f"Plotting {storm_name}.")  #e892c4
stat='density'
fig, axs = plt.subplots(1, 2, figsize=(20, 6.5), dpi=300, layout='constrained') # rows, cols, w, h
sns.histplot(pn_subset['max_sfc_wsp'].values.flatten(), stat=stat, element='step', color='#024588', alpha=0.0005, label='CAM5-SE', 
             kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[0])
sns.histplot(cn_subset['max_sfc_wsp'].values.flatten(), stat=stat, element='step', color='#BD0086', alpha=0.0005, label='CAM5-MPAS', 
             kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[0])
axs[0].set_title('Maximum Surface Wind Speed', fontsize=titles, fontweight='bold', loc='left')
axs[0].set_xlabel('Wind Speed [mph]', fontsize=labels)
axs[0].set_ylabel('Density', fontsize=labels)


g3 = sns.histplot(pn_subset['max_850_wsp'].values.flatten(), stat=stat, element='step', color='#024588', alpha=0.0005, label='CAM5-SE', 
             kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[1])
g4 = sns.histplot(cn_subset['max_850_wsp'].values.flatten(), stat=stat, element='step', color='#BD0086', alpha=0.0005, label='CAM5-MPAS', 
             kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[1])
axs[1].set_title('Maximum 850mb Wind Speed', fontsize=titles, fontweight='bold', loc='left')
axs[1].set_xlabel('Wind Speed [mph]', fontsize=labels)
axs[1].set_ylabel(None)

for ax in axs.ravel():
    ax.legend(fontsize=legends)
    ax.tick_params(axis='both', which='major', labelsize=ticks)

fig.savefig(f"../figs/{storms[j]}_wsp_histograms2.png", bbox_inches='tight', dpi=300)
plt.close()
#plt.show()